A partir de una tabla de datos ya procesada para utilizar con los modelos teoricos, 
en este nb buscamos optimizar

In [1]:
## seteamos los parametros del notebook
%autosave 180

Autosaving every 180 seconds


In [2]:
import pandas as pd; 
import numpy as np; 
import os;
import random;
import time
#%matplotlib inline

In [3]:
#from utils.partitions import *
#import utils.presto_connection as p_c

In [4]:
np.random.seed(2015)
#rootdir=os.getcwd()
rootdir = os.path.expanduser('~/Repositories/user-analytics/clustering')
os.chdir(rootdir)

#hdf5_file = "LTV_results.h5"
#store = pd.HDFStore(hdf5_file, mode='a', complevel = 9, complib='blosc')
run_number = 0

In [4]:
#from unbalanced_dataset import UnderSampler, NearMiss, CondensedNearestNeighbour, OneSidedSelection,\
#NeighbourhoodCleaningRule, TomekLinks, ClusterCentroids, OverSampler, SMOTE,\
#SMOTETomek, SMOTEENN, EasyEnsemble, BalanceCascade

# Data queries  Params
Para tener rapidamente parametrizadas las distintas queries y correrla en fault a mano o con Pyhive

## Dafiti  android 189
Dafiti me da mas informacion de "social sharing" events que tal vez me sirvan para ver el componente de virality en LTV.

Miramos key_events como
{363:"view_product",35:"sale",372:"product_rate",369:"social_share"}


### Lieferheld Case: android 889
Miramos como key_events: {2350:order_rate, 2296: in_app_acquisition, 2143: view_product}

### FoodPanda : android 545 e ios 544

**Obs** No tengo info de social_share en este advertiser ni info de product_rate

Miramos los key_events: {1149:"restaurant_view",184:"400_Add_to_Cart",187:"500_sale"}

### BoxedWholesale : android 1394 e ios  1395
**Obs** No tengo info de social_share en este advertiser ni info de product_rate **NI DE** product_view

Miramos key_events: {24:"open", 251: "add_to_cart", 8: "purchased"}

### Netshoes : android 1289 e ios 1325
**Obs:** No tengo info de social_share en este advertiser ni info de product_rate

Miramos key_events: key_evs = {1912:"product_view",251:"add_to_cart", 2393:"trackTransaction"}


In [5]:
# query params
year_start=2016; month_start=3;day_start=25;
year_end= 2016;month_end= 3;day_end= 31;
#event_id = 136

#key events de lieferheld 
#evs_ids = [2143,2350,2296]
#evs_names =["rt_view_product","order_rate","in_app_acqusition"]
#application_id=889; osp = "android"
#social_share = False

#key events de dafiti android 189
# tomo el perc count en 1e-3
evs_names =["view_listing","app_open","Search", "add_to_cart", "remove_from_cart", "add_to_wishlist"]
evs_ids = [362,372,366, 104, 90, 364, 368]
sale_event = 'sale'
sale_id = 35
application_id=189; osp ="android";
other_params = ['device_country','device_model','device_manufacturer',
                'device_countrycode','device_os_version','display_size','shop_countrycode']

#key events de netshoes 1289 android e ios 1325
#evs_names = ["product_view","add_to_cart", "trackTransaction"]
#evs_ids = [1912,251,2393]
#social_share = False
#application_id= 1289; osp="android";
#application_id= 1325; osp="ios";

# events de foodpandas android 545 ios 544 a mano
# tomo el perc en 1e-2
#evs_names =["Restaurant_view","Dish_view","add_To_Cart","restaurant_listing" ,"Launch" ,"Search_query" ,
#            "Voucher_checkout","remove_from_cart","Deal_view","Login"]
#evs_ids = [1149,778,184, 592,185 ,555, 186, 1704, 1709 ,189 ]
#sale_event = '500_sale'
#sale_id = 187
#social_share = False
#application_id= 545; osp="android";
#application_id= 544; osp="ios";
#other_params = ['device_countrycode','device_os_version',"price", "city"]

#key events de BoxedWholesale android 1394  ios 1395
#evs_names = ["open","add_to_cart","purchased"]
#evs_ids = [24, 251, 8]
#social_share = False
#application_id= 1394; osp="android";
#application_id= 1395; osp ="ios";

NameError: name 'get_os_partitions' is not defined

In [6]:
appid_name = {545:'foodpandas',189:'dafiti'}

In [7]:
train_file = "datasets/{name}_{ds:0=2d}{ms:0=2d}{de:0=2d}{me:0=2d}.csv.gz".format( ds=1,ms=3, 
             de = 15, me = 3,name = appid_name[application_id])
train_table = pd.read_csv(train_file,sep = "|",
                   header =0,
                     index_col =0)
for col in [col for col in train_table.columns if ("count_" in col )]:
    train_table[col] = train_table[col].astype(int)
    
for col in [col for col in train_table.columns if  "time_" in col]:
    train_table[col] = train_table[col].astype(float)

X_train = train_table[train_table.columns[:-1]].values
y_train = train_table[train_table.columns[-1:]].values.flatten()

In [8]:
test_file = "datasets/{name}_{ds:0=2d}{ms:0=2d}{de:0=2d}{me:0=2d}.csv.gz".format(  ds=15,ms=3,
            de = 31, me = 3,name=appid_name[application_id])
test_table = pd.read_csv(test_file,sep = "|",
                   header =0,
                     index_col =0)
for col in [col for col in test_table.columns if ("count_" in col )]:
    test_table[col] = test_table[col].astype(int)
    
for col in [col for col in test_table.columns if  "time_" in col]:
    test_table[col] = test_table[col].astype(float)

X_test = test_table[test_table.columns[:-1]].values
y_test = test_table[test_table.columns[-1:]].values.flatten()

test_table.reset_index(inplace=True)
test_table = test_table[['ref_hash','y']]

## Load sklearn utlities

In [19]:
import sklearn

In [ ]:
sklearn.cross_validation.StratifiedKFold

In [20]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils import *
from sklearn.preprocessing import label_binarize, scale, StandardScaler
from sklearn.metrics import *
from sklearn.grid_search import *
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import StratifiedKFold

In [10]:
#elegimos algunas variantes del dataset original
scaler = StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)

perc = 0.2
mini_ind = np.random.choice(X_train.shape[0],int(perc*X_train.shape[0]),replace=False)
X_mini =  X_train[mini_ind,:]
y_mini = y_train[mini_ind]

In [19]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((354580, 281), (354580,), (371161, 281), (371161,))

## Set XGBoost 
Gradient boosting using xgboost dmlc tested library 
(google and kaggle.com)

In [12]:
#Parameter grid search with xgboost
#feature engineering is not so useful and the LB is so overfitted/underfitted
#so it is good to trust your CV
import xgboost as xgb
from sklearn import preprocessing
from sklearn.grid_search import *


msk = np.random.rand(len(X_train)) < 0.75

train = X_train[msk,:].copy()
train = np.c_[ y_train[msk], train ] 

test = X_train[~msk,:].copy()
test = np.c_[ y_train[~msk], test ] 


In [14]:
msk.shape, y_train[msk].shape, y_train[~msk].shape, X_train[msk,:].shape, X_train[~msk,:].shape

((354580,), (266121,), (88459,), (266121, 281), (88459, 281))

In [16]:

features = train_table.columns  #la colonne 0 est le quote_conversionflag  
print(features)


for f in range(train.shape[1]):
    if train_table[train_table.columns[f]].dtype=='object':
        print(train_table.columns[f])
        lbl = preprocessing.LabelEncoder()
        lbl.fit(np.r_[train[:,f] ,test[:,f]])
        train[:,f] = lbl.transform(list(train[:,f]))
        test[:,f] = lbl.transform(list(test[:,f]))


Index(['count_362_362', 'time_362_362', 'count_362_372', 'time_362_372',
       'count_362_366', 'time_362_366', 'count_362_104', 'time_362_104',
       'count_362_90', 'time_362_90',
       ...
       'display_size_6.2', 'display_size_6.8', 'display_size_7.0',
       'display_size_7.1', 'shop_country_AR', 'shop_country_BR',
       'shop_country_CL', 'shop_country_CO', 'shop_country_MX', 'y'],
      dtype='object', length=282)


In [50]:
#brute force scan for all parameters, here are the tricks

#usually max_depth is 6,7,8
#learning rate is around 0.05, but small changes may make big diff
#tuning min_child_weight subsample colsample_bytree can have 
#much fun of fighting against overfit 
#n_estimators is how many round of boosting
#finally, ensemble xgboost with multiple seeds may reduce variance
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate':list(10.0 ** np.arange(-3, 0)),
              'max_depth': [5,15,25],
              'reg_alpha': list(10.0 ** np.arange(-2, 1)),
              #'lambda': [1,0],
              'gamma': [1,2],
              'base_score': list(10.0 ** np.arange(-2, 0)),
              'gamma': list(10.0 ** np.arange(-2, 1)),

              'min_child_weight': [15,10,5,2],
              'silent': [1],
              'subsample': [0.8,0.5,0.3],
              'colsample_bytree':[1,0.8,0.5],
              'n_estimators': [30,50,80], #number of trees, change it to 1000 for better results
              #'missing':[-999],
              'n_estimators': [15,30],
              'seed': [1337]}

# cuento la cantidad de parametros que tengo
total =1
for key, value in parameters.items():
    total = total*len(value)
print(total)

11664


In [47]:
xgb_model.get_params().keys()

dict_keys(['base_score', 'gamma', 'reg_alpha', 'max_delta_step', 'objective', 'max_depth', 'reg_lambda', 'seed', 'scale_pos_weight', 'learning_rate', 'n_estimators', 'subsample', 'colsample_bytree', 'silent', 'nthread', 'min_child_weight', 'colsample_bylevel', 'missing'])

##  Best_Params
* learning_rate: betwween 0.01 to 0.1
* min_child_weight: >5 is better std error
* n_estimators: <30 is better std error
* max_depth: >15 is slightly slightly better in mean but slightly worse in std.


In [51]:
start_time = time.time()

xgb_model = xgb.XGBClassifier()

clf = GridSearchCV(xgb_model, parameters, n_jobs=3, 
                   scoring='f1',
                   verbose=2, refit=True)

clf.fit(X_train, y_train)

#trust your CV!
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

test_probs = clf.predict_proba(test[features])[:,1]

#sample = pd.read_csv('../input/sample_submission.csv')
#sample.QuoteConversion_Flag = test_probs
#sample.to_csv("xgboost_best_parameter_submission.csv", index=False)

all_time =   time.time() - start_time 
print('This cell took %s seconds to run' % all_time)

Fitting 3 folds for each of 11664 candidates, totalling 34992 fits


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] base_score=0.01, gamma=0.01, reg_alpha=0.01, n_estimators=15, objective=binary:logistic, max_depth=5, seed=1337, min_child_weight=15, learning_rate=0.001, subsample=0.8, colsample_bytree=1, silent=1, nthread=4 


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] base_score=0.01, gamma=0.01, reg_alpha=0.01, n_estimators=15, objective=binary:logistic, max_depth=5, seed=1337, min_child_weight=15, learning_rate=0.001, subsample=0.8, colsample_bytree=1, silent=1, nthread=4 


...

[CV]  base_score=0.01, gamma=0.01, reg_alpha=0.01, n_estimators=15, objective=binary:logistic, max_depth=25, seed=1337, min_child_weight=5, learning_rate=0.001, subsample=0.8, colsample_bytree=1, silent=1, nthread=4 -  54.6s
[CV] base_score=0.01, gamma=0.01, reg_alpha=0.01, n_estimators=15, objective=binary:logistic, max_depth=25, seed=1337, min_child_weight=5, learning_rate=0.001, subsample=0.5, colsample_bytree=1, silent=1, nthread=4 


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/metrics/classification.py:1117: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  base_score=0.01, gamma=0.01, reg_alpha=0.01, n_estimators=15, objective=binary:logistic, max_depth=25, seed=1337, min_child_weight=5, learning_rate=0.001, subsample=0.5, colsample_bytree=1, silent=1, nthread=4 - 1.1min
[CV] base_score=0.01, gamma=0.01, reg_alpha=0.01, n_estimators=15, objective=binary:logistic, max_depth=25, seed=1337, min_child_weight=5, learning_rate=0.001, subsample=0.3, colsample_bytree=1, silent=1, nthread=4 


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


KeyboardInterrupt: 

## Timings
Ahora si

In [ ]:
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05, 0.04, 1e-2,0.1,0.5], #so called `eta` value
              'max_depth': [4,5,6],
              'min_child_weight': [2,3,1],
              'silent': [1],
              'subsample': [0.75],
              'colsample_bytree': [0.7],
              'n_estimators': [30,50,80], #number of trees, change it to 1000 for better results
              #'missing':[-999],
              'seed': [42]}


In [62]:
predictions =  clf.predict(test[features])
expected = test['ground_truth'].values
print(classification_report(expected, predictions))

             precision    recall  f1-score   support

          0       0.95      1.00      0.97     32223
          1       0.38      0.00      0.00      1772

avg / total       0.92      0.95      0.92     33995



In [88]:
predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(test[features])[:,1]
predicted_proba['gt'] = test['ground_truth'].values
predicted_proba['class'] = clf.predict(test[features])

threshold = 1e-1
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))

In [91]:
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1

In [92]:
cv_result

,mean_score,colsample_bytree,silent,learning_rate,nthread,min_child_weight,n_estimators,subsample,seed,objective,max_depth
0,0.734251,0.7,1.0,0.05,3.0,11.0,30.0,0.75,1337.0,binary:logistic,10.0
1,0.733508,0.7,1.0,0.05,3.0,11.0,50.0,0.75,1337.0,binary:logistic,10.0
2,0.733174,0.7,1.0,0.05,3.0,11.0,80.0,0.75,1337.0,binary:logistic,10.0
3,0.734326,0.7,1.0,0.05,3.0,5.0,30.0,0.75,1337.0,binary:logistic,10.0
4,0.733291,0.7,1.0,0.05,3.0,5.0,50.0,0.75,1337.0,binary:logistic,10.0
5,0.732507,0.7,1.0,0.05,3.0,5.0,80.0,0.75,1337.0,binary:logistic,10.0
6,0.734111,0.7,1.0,0.05,3.0,1.0,30.0,0.75,1337.0,binary:logistic,10.0
7,0.733195,0.7,1.0,0.05,3.0,1.0,50.0,0.75,1337.0,binary:logistic,10.0
8,0.732155,0.7,1.0,0.05,3.0,1.0,80.0,0.75,1337.0,binary:logistic,10.0
9,0.734849,0.7,1.0,0.05,3.0,11.0,30.0,0.75,1337.0,binary:logistic,5.0


In [113]:
coln = 1
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
cv_result.groupby(cv_result.columns[coln])['mean_score'].std()

colsample_bytree
0.7    0.726528
Name: mean_score, dtype: float64


colsample_bytree
0.7    0.015647
Name: mean_score, dtype: float64

##  Best_Params
* learning_rate: betwween 0.01 to 0.1
* min_child_weight: >5 is better std error
* n_estimators: <30 is better std error
* max_depth: >15 is slightly slightly better in mean but slightly worse in std.


# Todo
* tune xgboost and libffm
